In [ ]:
import numpy as np
import nrrd
from os.path import *
from pylab import *
from JSONread import *
from density_function import *
import json
import xlrd

DATA_FOLDER = "data/"
OUTPUT_FOLDER = "output/"

In [ ]:
annotation, h = nrrd.read(join(DATA_FOLDER, "annotations.nrrd"))
neu_dens, h = nrrd.read(join(DATA_FOLDER, "neu_density.nrrd"))
num_neurons = json.loads(open(join(DATA_FOLDER, "neuron_counts.json"), "r").read())
jsontextfile = open(join(DATA_FOLDER, "brain_regions.json"), "r")
jsoncontent = json.loads(jsontextfile.read())
search_children(jsoncontent['msg'][0])
rv = json.loads(open(join(DATA_FOLDER, "volumes_25.json"), "r").read())
voxel_volume = 25**3/1.0e9

# Computing densities

In [ ]:
uniques = find_unique_regions(annotation, 
                        id_to_region_dictionary_ALLNAME, 
                        region_dictionary_to_id_ALLNAME,
                        region_dictionary_to_id_ALLNAME_parent, 
                        name2allname)

children, order_ = find_children(uniques, id_to_region_dictionary_ALLNAME, is_leaf,
                                  region_dictionary_to_id_ALLNAME_parent, 
                                  region_dictionary_to_id_ALLNAME)
uniques = uniques[np.argsort(order_)] # order from leaf to biggest regions

In [ ]:
####### Choose Markers and their files
marker_names = ["PV", "SST", "VIP"] # Careful with the order: should work with the literature file
volume_files = ["868_pvalb_expr.npy", "1001_SST_expr.npy", "77371835_VIP_expr.npy"]
literature_file = join(DATA_FOLDER, "densities.xlsx")
sheet_indices = [0]
num_first_row = 2
column_name = 1
columns_mean = [[3, 5], [7, 9], [11, 13]]
num_marker = len(marker_names)

In [ ]:
markers_intensity = np.zeros((num_marker,) + annotation.shape)
for i_marker, filename in enumerate(volume_files):
    markers_intensity[i_marker] = np.load(join(DATA_FOLDER, filename))

In [ ]:
# Loading results of the fitting pipeline
alphas = np.zeros((num_marker, 3))
std_fitt = np.zeros((num_marker, 3))
for i_marker, name in enumerate(marker_names):
    jsoncontent = json.loads(open(join(OUTPUT_FOLDER, "fitting_" + name + ".json"), "r").read())
    for i_key, key in enumerate(["Cerebellum", "Isocortex", "Rest"]):
        if key in jsoncontent["alphas"].keys():
            alphas[i_marker][i_key] = jsoncontent["alphas"][key]
            std_fitt[i_marker][i_key] = jsoncontent["std"][key]

In [ ]:
names, mean_literature, down_std_literature, up_std_literature, convert = read_densities_sheet(
                    literature_file, region_keys, columns_mean,
                    sheet_indices, num_first_row, column_name)

In [ ]:
# Conversion dictionary from Kim to CCF
convert["Whole brain"] = ["Basic cell groups and regions"]
convert["Anterior cingulate area, ventral part, 6"] = ["Anterior cingulate area, layer 6a", "Anterior cingulate area, layer 6b"]

# Uncomment for CCFv3, corrections for regions which were in CCFv2
# for old, new in dict_corrections.items():
#     old_name = id_to_region_dictionary[new[0]]+'/3'
#     if old_name in convert.keys():
#         convert[old_name] = [id_to_region_dictionary[inew] for inew in new]

for id_, name in id_to_region_dictionary.items():
    if "layer 6" in name or "Layer 6" in name:
        if name[:-1] in convert.keys():
            convert[name[:-1]].append(name)

invert_convert = {}
for k, vs in convert.items():
    for v in vs:
        invert_convert[v] = k
# Regions not found: regions which are merged in CCF mostly
print("Regions without CCF equivalent:")
for k,v in convert.items():
    if len(v)==0:
        print(k)
        
for i_region, name in enumerate(names):
    if name in invert_convert.keys():
        names[i_region] = invert_convert[name]

In [ ]:
# Pure non inh regions 
wb2 = xlrd.open_workbook(join(DATA_FOLDER, "gaba_papers.xlsx"))
sheet2 = wb2.sheet_by_index(1)
ids_pure = []
for i_region in range(sheet2.nrows-1): 
    name = sheet2.cell_value(i_region+1, 0)
    if name in region_keys:
        value = sheet2.cell_value(i_region+1, 1)
        if value<1e-6:
            id_reg = region_dictionary_to_id[name]
            ids_pure.append(id_reg)
uniques = uniques[np.in1d(uniques, ids_pure, invert=True)]

In [ ]:
# Principal loop, should take a while
dens_markers, std_markers = place_cells(annotation, uniques, children, 
                                        rv, neu_dens, num_neurons,
                                        markers_intensity, alphas, std_fitt, 
                                        names, mean_literature, down_std_literature, up_std_literature,
                                        id_to_region_dictionary_ALLNAME, region_dictionary_to_id_ALLNAME,
                                        is_leaf, id_to_region_dictionary)

In [ ]:
for i_marker in range(num_marker):
    for id_reg in ids_pure:
        std_markers[i_marker][str(id_reg)] = [0., 0.]

In [ ]:
dens_markers /= voxel_volume
for i_marker, name in enumerate(marker_names):
    nrrd.write(join(OUTPUT_FOLDER, "densities_" + name + ".nrrd"), dens_markers[i_marker], header=h)
    with open(join(OUTPUT_FOLDER, "std_" + name + ".json"), 'w') as fp:
        json.dump(std_markers[i_marker], fp, indent=4)

# Plot results

In [ ]:
dens_PV, h = nrrd.read(join(OUTPUT_FOLDER, "densities_PV.nrrd"))
dens_SST, h = nrrd.read(join(OUTPUT_FOLDER, "densities_SST.nrrd"))
dens_VIP, h = nrrd.read(join(OUTPUT_FOLDER, "densities_VIP.nrrd"))

In [ ]:
figure(figsize=(20,15))
imshow(dens_PV[250,:,:], cmap='hot')
colorbar()
show()

In [ ]:
reg_names = ["Primary visual area", "Primary somatosensory area, lower limb", "Prelimbic area", "Primary somatosensory area, barrel field"]
colors = ["darkblue", "green", "orange", "red"]
names = ["L1", "L2","L3","L4","L5","L6a","L6b"]
PVs =[]
PVs_std =[]
SSTs =[]
SSTs_std =[]
VIPs =[]
VIPs_std =[]
figure(figsize=(20,10))
for ireg, reg in enumerate(reg_names):
    dens_out_PV = np.zeros(7)
    dens_std_PV = np.zeros(7)
    dens_out_SST = np.zeros(7)
    dens_std_SST = np.zeros(7)
    dens_out_VIP = np.zeros(7)
    dens_std_VIP = np.zeros(7)
    ids_roi = children[name2allname[reg]]
    for id_child in ids_roi:
        pos = -1
        filter_ = annotation==id_child
        name = id_to_region_dictionary[id_child]
        if "Rostrolateral" in name:
            continue
        if "ayer 1" in name:
            pos = 0
        elif "ayer 2" in name:
            pos = 1
        elif "ayer 3" in name:
            pos = 2
        elif "ayer 4" in name:
            pos = 3
        elif "ayer 5" in name:
            pos = 4
        elif "ayer 6a" in name:
            pos = 5
        elif "ayer 6b" in name:
            pos = 6
        dens_out_PV[pos]=np.mean(dens_PV[filter_])
        dens_std_PV[pos]=np.std(dens_PV[filter_])
        dens_out_SST[pos]=np.mean(dens_SST[filter_])
        dens_std_SST[pos]=np.std(dens_SST[filter_])
        dens_out_VIP[pos]=np.mean(dens_VIP[filter_])
        dens_std_VIP[pos]=np.std(dens_VIP[filter_])
    PVs.append(dens_out_PV)
    PVs_std.append(dens_std_PV)
    SSTs.append(dens_out_SST)
    SSTs_std.append(dens_std_SST)
    VIPs.append(dens_out_VIP)
    VIPs_std.append(dens_std_VIP)    
    ax1 = subplot2grid((4,3), (ireg,0), colspan=1, rowspan=1)
    ax1.bar(names, dens_out_PV, yerr=dens_std_PV, color=colors[ireg])
    ax1.set_title("PV layer densities for "+ reg)
    ax1.set_xticks(names)
    ax1 = subplot2grid((4,3), (ireg,1), colspan=1, rowspan=1)
    ax1.bar(names, dens_out_SST, yerr=dens_std_SST, color=colors[ireg])
    ax1.set_title("SST layer densities for "+ reg)
    ax1.set_xticks(names)
    ax1 = subplot2grid((4,3), (ireg,2), colspan=1, rowspan=1)
    ax1.bar(names, dens_out_VIP, yerr=dens_std_VIP, color=colors[ireg])
    ax1.set_title("VIP layer densities for "+ reg)
    ax1.set_xticks(names)
tight_layout()
savefig(join(OUTPUT_FOLDER,"final_densities.png"))